# Transformers

## Módulos

In [1]:
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, 
    Trainer, TrainingArguments
)
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, 
    Trainer, TrainingArguments
)
from transformers.trainer_callback import PrinterCallback
from transformers.utils.notebook import NotebookProgressCallback
import numpy as np
import pandas as pd
import torch
from datasets import Dataset

/home/pauli/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-16 19:41:42.697479: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 19:41:42.914390: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-16 19:41:42.914401: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your 

In [43]:
from utils.json_utils import read_json, write_json

## Corpus

In [3]:
corpus = read_json("1_scrapping_black_clover.json")
len(corpus)

163

In [4]:
corpus[0]

"In Hage, a priest finds two babies abandoned outside his church. He takes them inside and discovers their names to be Yuno and Asta.\nFifteen years later, Asta proposes to Sister Lily, who refuses repeatedly. Yuno and the other orphans criticize Asta and point out his lack of magic. Asta tries to show off his skills, but Yuno outshines him with his magic.\nLater, at the Grimoire Acceptance Ceremony, a pair of nobles criticize the commoners there. Despite Asta not receiving a grimoire while Yuno attains a four-leaf clover one, Asta challenges Yuno to the title of Wizard King, but Yuno ignores him.\nAfter the ceremony, the two nobles ambush Yuno outside the tower, but Yuno overpowers them. All three are then ambushed by Revchi Salik, a former Magic Knight who plans to steal Yuno's grimoire and sell it. Asta comes to Yuno's rescue but fails.\nAs Revchi breaks Asta's spirit, Yuno calls Asta his rival. Motivated by this acknowledgment, Asta decides to keep fighting, which summons a grimoir

In [5]:
aux = []
for doc in corpus:
    sentences = doc.split("\n")
    for sentence in sentences:
        if len(sentence) > 1024:
            cut_sentence = sentence.split(".")
            aux.extend(cut_sentence)
        else:
            aux.append(sentence)
corpus = aux
len(corpus)

2154

In [6]:
corpus[0]

'In Hage, a priest finds two babies abandoned outside his church. He takes them inside and discovers their names to be Yuno and Asta.'

In [7]:
max_length = 0
for doc in corpus:
    doc_len    = len(doc)
    max_length = doc_len if doc_len > max_length else max_length
    if doc_len > 1024:
        print(doc, "\n\n")
max_length

1021

In [8]:
eps1_3 = corpus[:3]

## Tokenización

In [9]:
model_checkpoint = "distilgpt2"

In [10]:
# Documentación:
# https://huggingface.co/docs/transformers/main_classes/tokenizer#tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [11]:
outputs_ep1_3 = tokenizer(
    eps1_3,
    truncation=True,
    max_length=max_length,
    return_length=True
)
outputs_ep1_3["length"]

[30, 53, 59]

In [12]:
outputs_ep1_3

{'input_ids': [[818, 367, 496, 11, 257, 11503, 7228, 734, 11903, 9958, 2354, 465, 4928, 13, 679, 2753, 606, 2641, 290, 27472, 511, 3891, 284, 307, 20757, 78, 290, 8304, 64, 13], [44403, 7821, 812, 1568, 11, 8304, 64, 26017, 284, 26553, 20037, 11, 508, 17567, 7830, 13, 20757, 78, 290, 262, 584, 50213, 24628, 8304, 64, 290, 966, 503, 465, 3092, 286, 5536, 13, 8304, 64, 8404, 284, 905, 572, 465, 4678, 11, 475, 20757, 78, 503, 1477, 1127, 683, 351, 465, 5536, 13], [18602, 11, 379, 262, 43077, 21699, 590, 26532, 9926, 11, 257, 5166, 286, 42173, 24628, 262, 2219, 364, 612, 13, 7945, 8304, 64, 407, 6464, 257, 18288, 32177, 981, 20757, 78, 708, 1299, 257, 1440, 12, 33201, 537, 2502, 530, 11, 8304, 64, 6459, 20757, 78, 284, 262, 3670, 286, 16884, 2677, 11, 475, 20757, 78, 24245, 683, 13]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [13]:
tokens_ep1 = tokenizer.tokenize(eps1_3[0]) 
print(tokens_ep1)

['In', 'ĠH', 'age', ',', 'Ġa', 'Ġpriest', 'Ġfinds', 'Ġtwo', 'Ġbabies', 'Ġabandoned', 'Ġoutside', 'Ġhis', 'Ġchurch', '.', 'ĠHe', 'Ġtakes', 'Ġthem', 'Ġinside', 'Ġand', 'Ġdiscovers', 'Ġtheir', 'Ġnames', 'Ġto', 'Ġbe', 'ĠYun', 'o', 'Ġand', 'ĠAst', 'a', '.']


In [14]:
tokenize_fn = lambda doc : tokenizer(
        doc,
        truncation=True,
        max_length=max_length
    )

tokenized_dataset = list(map(tokenize_fn, corpus))

In [15]:
tokenized_dataset = pd.DataFrame(tokenized_dataset)
tokenized_dataset = Dataset.from_pandas(tokenized_dataset)

In [16]:
tokenized_dataset["input_ids"]

[[818,
  367,
  496,
  11,
  257,
  11503,
  7228,
  734,
  11903,
  9958,
  2354,
  465,
  4928,
  13,
  679,
  2753,
  606,
  2641,
  290,
  27472,
  511,
  3891,
  284,
  307,
  20757,
  78,
  290,
  8304,
  64,
  13],
 [44403,
  7821,
  812,
  1568,
  11,
  8304,
  64,
  26017,
  284,
  26553,
  20037,
  11,
  508,
  17567,
  7830,
  13,
  20757,
  78,
  290,
  262,
  584,
  50213,
  24628,
  8304,
  64,
  290,
  966,
  503,
  465,
  3092,
  286,
  5536,
  13,
  8304,
  64,
  8404,
  284,
  905,
  572,
  465,
  4678,
  11,
  475,
  20757,
  78,
  503,
  1477,
  1127,
  683,
  351,
  465,
  5536,
  13],
 [18602,
  11,
  379,
  262,
  43077,
  21699,
  590,
  26532,
  9926,
  11,
  257,
  5166,
  286,
  42173,
  24628,
  262,
  2219,
  364,
  612,
  13,
  7945,
  8304,
  64,
  407,
  6464,
  257,
  18288,
  32177,
  981,
  20757,
  78,
  708,
  1299,
  257,
  1440,
  12,
  33201,
  537,
  2502,
  530,
  11,
  8304,
  64,
  6459,
  20757,
  78,
  284,
  262,
  3670,
  286,
  16884,
  

In [17]:
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint, pad_token_id=tokenizer.eos_token_id)

## Entrenamiento

In [18]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [19]:
out = data_collator([tokenized_dataset[i] for i in range(3)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


attention_mask shape: torch.Size([3, 59])
input_ids shape: torch.Size([3, 59])
labels shape: torch.Size([3, 59])


In [20]:
out["input_ids"][1]

tensor([44403,  7821,   812,  1568,    11,  8304,    64, 26017,   284, 26553,
        20037,    11,   508, 17567,  7830,    13, 20757,    78,   290,   262,
          584, 50213, 24628,  8304,    64,   290,   966,   503,   465,  3092,
          286,  5536,    13,  8304,    64,  8404,   284,   905,   572,   465,
         4678,    11,   475, 20757,    78,   503,  1477,  1127,   683,   351,
          465,  5536,    13, 50256, 50256, 50256, 50256, 50256, 50256])

In [21]:
out["attention_mask"][1]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])

In [22]:
model_name = model_checkpoint.split("/")[-1] 
model_name

'distilgpt2'

In [23]:
training_args = TrainingArguments(
    f"{model_name}-finetuned-black-clover",
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,    
    learning_rate=2e-5,
    weight_decay=0.01,
    do_eval=True, # eval en validation set
    evaluation_strategy="steps", # eval en validation set
    eval_steps=100,
    save_steps=100, # checkpoint model every 500 steps
    logging_dir='./logs', # logging
    logging_strategy="steps",
    logging_steps=1,
    fp16=False, # float16 en training (only on CUDA)
    push_to_hub=False,
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset, #.select(range(0, 128)),
    eval_dataset=tokenized_dataset, #.select(range(0, 128)),
)

In [25]:
!rm -rf ./logs

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
%reload_ext tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6007 (pid 9162), started 2:15:25 ago. (Use '!kill 9162' to kill it.)

In [27]:
train_output = trainer.train()

/home/pauli/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2154
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 680
  0%|          | 1/680 [00:18<3:30:28, 18.60s/it]

{'loss': 4.5172, 'learning_rate': 1.997058823529412e-05, 'epoch': 0.01}


  0%|          | 2/680 [00:35<3:21:48, 17.86s/it]

{'loss': 4.6805, 'learning_rate': 1.9941176470588238e-05, 'epoch': 0.03}


  0%|          | 3/680 [00:56<3:34:30, 19.01s/it]

...

{'loss': 3.1808, 'learning_rate': 5.882352941176471e-08, 'epoch': 9.97}


100%|█████████▉| 679/680 [3:43:58<00:18, 18.20s/it]

{'loss': 3.2011, 'learning_rate': 2.9411764705882354e-08, 'epoch': 9.99}


100%|██████████| 680/680 [3:44:03<00:00, 14.16s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 680/680 [3:44:03<00:00, 19.77s/it]

{'loss': 3.3526, 'learning_rate': 0.0, 'epoch': 10.0}
{'train_runtime': 13443.2501, 'train_samples_per_second': 1.602, 'train_steps_per_second': 0.051, 'train_loss': 3.517763341174406, 'epoch': 10.0}


In [28]:
trainer.save_model()

Saving model checkpoint to distilgpt2-finetuned-black-clover
Configuration saved in distilgpt2-finetuned-black-clover/config.json
Model weights saved in distilgpt2-finetuned-black-clover/pytorch_model.bin


## Evaluación

In [29]:
val_results = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2154
  Batch size = 32
100%|██████████| 68/68 [05:30<00:00,  4.85s/it]


In [30]:
val_results

{'eval_loss': 3.21470046043396,
 'eval_runtime': 334.8291,
 'eval_samples_per_second': 6.433,
 'eval_steps_per_second': 0.203,
 'epoch': 10.0}

## Generación de Texto

In [31]:
device = f"cuda:{torch.cuda.current_device()}" if torch.cuda.is_available() else "cpu"

In [32]:
def generate(
    prompt=None, max_length=100, greedy=True, model=model, tokenizer=tokenizer, device=device
):
    """None stands for beggining of sequence.
    NOTE si bien parece que GPT2 puede generar a partir de BOS token, la 
    documentacion es poco clara. Ademas hicimos nuestro finetuning sin BOS token.
    Entonces solo vamos a usar la funcion pasandole un contexto.

    Ver:
    https://github.com/huggingface/transformers/issues/3311#issuecomment-601264426
    https://github.com/openai/gpt-2/blob/a74da5d99abaaba920de8131d64da2862a8f213b/src/generate_unconditional_samples.py#L60
    """
    do_sample = False if greedy else True
    # model.eval() to set dropout and batch normalization layers to evaluation 
    # mode before running inference
    if prompt:
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
        model.eval()
        outputs = model.generate(input_ids, do_sample=do_sample, max_length=max_length)
    else:
        model.eval()
        outputs = model.generate(do_sample=do_sample, max_length=max_length)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [39]:
res_ = generate("Yuno can be described as", greedy=False)
print(res_[0])

Yuno can be described as arrogant and weak, strong and has never beaten a man. Licht hopes to save Asta as well, but that he cannot beat someone at a fast enough. Licht offers to help him but Yami decides to go to help him. Yami warns him to go head to head with the Devil Devil to ensure the end of the story is to come. As he flies away from the Devil's base, Leopold informs Yami of what he's going


In [40]:
characters = read_json("3_characters_black_clover.json")

In [41]:
phrase2desc = " can be described as"
character_descs = dict()
for character in characters:
    character_descs[character] = []
    for i in range(0, 10):
        res = generate(character + phrase2desc, greedy=False)
        character_descs[character].append(res)
character_descs

{'Acier': [['Acier can be described as having a similar appearance to Gauche Vermillion but with a slightly longer torso and shorter torso and a longer torso. He also has a hair-length scar with an intense red scar on his fingers and is covered in blood. Gauche also appears to be a woman in a black dress and wearing a pink dress. Gauche explains to him that he was only able to go on his adventures and to prove that he was one of the most fun people he had ever met'],
  ['Acier can be described as a kind of Magic Magic Knight and uses his magic to attack his teammates with his Dark Rises, which can be much stronger than it was for the Dark Knights. Roulacase and Noelle Silva also appear. Silva reveals that Asta took their form because they were all orphans. Asta asks them where they lived but he tells them to go, saying that they are the orphans only. Asta asks where they live and, like Asta, tells him'],
  ['Acier can be described as possessing both of the same ability and can sense ki

In [44]:
write_json(character_descs, "13_character_descriptions_black_clover.json")